# 4 Using LLMs as Judges

Previous sections in this lab all focus on easy to evaluate outputs: single word, multiple choice, true/false etc.  The evaluation of longer text (summary) only uses NLP metrics which might not be too related to its semantics.  In order to better understand the semantics, and other "quality" of the answer, we can use a (more powerful) LLM as a judge. 

In this subsection, we'll explore how to use LLMs as judges to evaluate model outputs. This approach leverages the capabilities of large language models to assess the quality of responses from other models, providing a flexible evaluation method that can be applied across various tasks. And llm-as-judge is also a well-used method in industry and LLM research field now. For more detailed knowledge, I recommand you to read this survey: https://arxiv.org/pdf/2411.15594?

Here, we provide a simplified process and discusses several common challenges to llm-as-judge.

## 4.1 Setting up

We use the same task as 01_evaluation, i.e., CNN_DAILYMAIL here.

In [7]:
#Setting up the environment
#Continue to use OpenAI API and INFINI Platform
from dotenv import load_dotenv
import os
import threading
from openai import OpenAI
import time
import random
from openai import RateLimitError


In [8]:
# we use API in this section
load_dotenv()
openai_api_key = os.environ.get("INFINI_API_KEY")
openai_base_url = os.environ.get("INFINI_BASE_URL")
client = OpenAI(api_key=openai_api_key, base_url=openai_base_url)

In [9]:
# these are just duplicates from section 2.  You can skip them if you have already read section 2.

def get_llm_output(model_name, question_content, question_id, output, semaphore=None):
    try:
        retries = 5
        for attempt in range(retries):
            try:
                if isinstance(question_content, str):
                    messages = [{"role": "user", "content": question_content}]
                else:
                    messages = [{"role": "user", "content": q} for q in question_content]

                chat_response = client.chat.completions.create(
                    model=model_name,
                    messages=messages,
                    max_tokens=2048,
                    temperature=0,
                )

                llm_answer = chat_response.choices[0].message.content.strip()
                output[question_id] = llm_answer
                break  # 成功就退出 retry 循环

            except RateLimitError as e:
                wait_time = random.uniform(1, 3) * (2 ** attempt)
                print(f"RateLimitError on Q{question_id}, retry {attempt+1}/{retries}, wait {wait_time:.1f}s")
                time.sleep(wait_time)

            except Exception as e:
                print(f"Error on Q{question_id}: {e}")
                break 

        else:
            output[question_id] = ""

    finally:
        if semaphore:
            semaphore.release()

def get_llm_output_parallel(model_name, question_contents, max_threads=5):
    # Create threads for each question
    output = {}
    threads = []
    semaphore = threading.Semaphore(max_threads)
    for question_id, question_content in tqdm(enumerate(question_contents)):
        semaphore.acquire() 
        thread = threading.Thread(target=get_llm_output, args=(model_name, question_content, question_id, output, semaphore))
        threads.append(thread)
        thread.start()

    # Wait for all threads to complete
    for thread in threads:
        thread.join()

    sorted_keys = sorted(output.keys())
    sorted_outputs = [output[key] for key in sorted_keys]        
    return sorted_outputs


Now let's load the dataset.  The dataset is described as

1. Source:
- Created from articles published on CNN and Daily Mail websites
- Articles were collected between 2007 and 2015
2. Each example consists of:
- A news article (document)
- Bullet point highlights/summaries of the article
- The highlights are treated as reference summaries

The dataset is popular in evaluating summarization tasks. 

In [4]:
from datasets import load_dataset, load_from_disk

d = load_from_disk('/ssdshare/share/data/cnn_dailymail/')

In [ ]:
def create_prompt(x):
    s = "Please summarize the following news article in no more than 30 words.\n %s" %(x['article'])
    x['question_content'] = s
    return x

d = d['test'].map(create_prompt) # Use test set to evaluate
d

## 4.2 The basic judge model (Direct Scoring)

 
We'll first generate summaries using different models, then have a judge model evaluate these summaries, and finally analyze the results to determine which model performs better at the summarization task, generating a radar graph.


In [6]:
# let's create a judge model first (by prompting it)

import json
import os
from tqdm import tqdm
import re

# Here is the prompt for the judge model
def create_judge_prompt(article, reference, model_answer, model_name):
    prompt = f"""Please evaluate the following summary of a news article. 
    
Original Article:
{article}

# Reference Summary:
# {reference}

Model ({model_name}) Summary:
{model_answer}

Please rate the model's summary on the following dimensions on a scale of 1-20 (1 being the not good one, 20 being the perfect one):
1. Elegance: Comparing to the reference summary (which is 10 points), how well-written and stylistically pleasing is the summary?
2. Fluency: Comparing to the reference summary (which is 10 points), how grammatically correct and easy to read is the summary?
3. Conciseness: Comparing to the reference summary (which is 10 points), how efficiently does the summary convey the key information?
4. Creativity: Comparing to the reference summary (which is 10 points), how original and innovative is the summary compared to a standard summary?

For each dimension, provide a score and a brief explanation. Format your response as follows:
{{
  "elegance": {{
    "score": <score>,
    "explanation": "<explanation>"
  }},
  "fluency": {{
    "score": <score>,
    "explanation": "<explanation>"
  }},
  "conciseness": {{
    "score": <score>,
    "explanation": "<explanation>"
  }},
  "creativity": {{
    "score": <score>,
    "explanation": "<explanation>"
  }}
}}
"""
    return prompt

def parse_judge_output(judge_output):
    # this inherited function is widely used in research repositories
    # it does some simple fixes to the output to make it a valid JSON
    try:
        # 尝试提取花括号包裹的部分
        match = re.search(r"\{.*\}", judge_output, re.DOTALL)
        if not match:
            raise ValueError("No JSON object found in the output.")
        
        json_str = match.group(0)

        # 简单修复常见 JSON 错误：缺逗号、缺括号等
        json_str = re.sub(r'("explanation"\s*:\s*"[^"]+)"\s*(")', r'\1,\2', json_str)  # 补逗号
        json_str = json_str.strip()

        # 尝试解析 JSON
        return json.loads(json_str)

    except Exception as e:
        print(f"Error parsing judge output: {e}")
        print(f"Raw output: {judge_output}")
        return {
            "elegance": {"score": 10, "explanation": "Failed to parse"},
            "fluency": {"score": 10, "explanation": "Failed to parse"},
            "conciseness": {"score": 10, "explanation": "Failed to parse"},
            "creativity": {"score": 10, "explanation": "Failed to parse"}
        }

# Create output directory if it doesn't exist
# note that because the output is expensive to get, better to save it to a file so we can examine it later
output_dir = "~/judge_results"  # avoid putting it into the current directory (poputing your codebase)
os.makedirs(output_dir, exist_ok=True)


In [7]:
import os
import json
from tqdm import tqdm

def evaluate_model(
    judge_model: str,
    model_name: str,
    dataset: dict,
    evaluate_n: int = 10,
    output_dir: str = output_dir,
    model_threads: int = 5,
) -> dict:


    # 1. take a subset of the dataset
    articles   = dataset['article'][:evaluate_n]
    questions  = dataset['question_content'][:evaluate_n]
    references = dataset['highlights'][:evaluate_n]

    # 2. ask the model to generate answers
    model_answers = get_llm_output_parallel(
        model_name, questions, max_threads=model_threads
    )

    results = []

    # 3. generate a score using the judge model
    for i in tqdm(range(evaluate_n), desc=f"Judging {model_name}"):
        judge_prompt  = create_judge_prompt(
            articles[i], references[i], model_answers[i], model_name
        )
        judge_response = get_llm_output_parallel(
            judge_model, [judge_prompt], max_threads=1
        )[0]

        scores = parse_judge_output(judge_response)

        results.append({
            "article_id"  : dataset['id'][i],
            "article"     : articles[i],
            "reference"   : references[i],
            "model_answer": model_answers[i],
            "scores"      : scores
        })

    # 4. save the results to a file
    os.makedirs(output_dir, exist_ok=True)
    output_file = os.path.join(
        output_dir, f"{model_name}_evaluations_{judge_model}.jsonl"
    )
    with open(output_file, "w", encoding="utf-8") as f:
        for r in results:
            f.write(json.dumps(r, ensure_ascii=False) + "\n")

    # 5. compute the average scores
    dims = ["elegance", "fluency", "conciseness", "creativity"]
    avg_scores = {d: sum(r["scores"][d]["score"] for r in results) / evaluate_n
                  for d in dims}

    # 6. print the results (optional)
    print(f"\n{model_name} average scores (judged by {judge_model}):")
    for d, s in avg_scores.items():
        print(f"  {d:<12}: {s:.2f}")
    print(f"\nResults saved to {output_file}")

    return {
        "output_file": output_file,
        "avg_scores" : avg_scores,
    }


In [ ]:
# Use 10 samples for evaluation to save time.
evaluate_n = 10

judge_model = "deepseek-v3"
model_names = ["qwen2.5-7b-instruct", "llama-2-7b-chat", "deepseek-v3"]

for model_name in model_names:
    evaluate_model(judge_model, model_name, d, evaluate_n)


In [9]:
import os, json, numpy as np
import matplotlib.pyplot as plt

# utility function to plot the results
def plot_model_comparison_radar(model_names, judge_model, output_dir=output_dir):

    all_avg_scores = {}
    for model_name in model_names:
        path = os.path.join(output_dir,
                            f"{model_name}_evaluations_{judge_model}.jsonl")
        if not os.path.exists(path):
            raise FileNotFoundError(f"Result file not found: {path}")
        
        with open(path, "r", encoding="utf-8") as f:
            results = [json.loads(line) for line in f]
        
        # Initialize the dimensions
        dims = ["elegance", "fluency", "conciseness", "creativity"]
        avg = {d: 0.0 for d in dims}
        for r in results:
            for d in dims:
                avg[d] += r["scores"][d]["score"]
        for d in dims:
            avg[d] /= len(results)
        all_avg_scores[model_name] = avg
    
    dimensions = list(next(iter(all_avg_scores.values())).keys())
    N = len(dimensions)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # close the circle
    
    fig, ax = plt.subplots(figsize=(8, 6), subplot_kw=dict(polar=True))
    
    plt.xticks(angles[:-1], dimensions, size=12)
    
    ax.set_rlabel_position(0)
    plt.yticks([4, 8, 12, 16, 20], ["4", "8", "12", "16", "20"], color="grey", size=10)
    plt.ylim(0, 20)
    
    for model_name in model_names:
        vals = [all_avg_scores[model_name][d] for d in dimensions]
        vals += vals[:1]
        ax.plot(angles, vals, linewidth=2, linestyle='solid', label=model_name)
        ax.fill(angles, vals, alpha=0.1)
    
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(f"LLM Judge Evaluation (Judge: {judge_model})", size=14, y=1.08)
    plt.tight_layout()
    plt.show()



In [ ]:
plot_model_comparison_radar(
    model_names=model_names,
    judge_model=judge_model
)

## 4.3 Bias1: Same family models

The researchers found that llm as judge had very some problems, such as the model with lower ability sometimes cannot be a good judge, and the model's tendency to give higher scores to the answers of its own or model from same 'famliy'. We will discuss and investigate this in this section.

In [ ]:
# let's test model's bias here
# let's first use qwen2.5 model as judge
judge_model = "qwen2.5-7b-instruct"
model_names = ["qwen2.5-72b-instruct", "llama-3.3-70b-instruct", "deepseek-v3"]
for model_name in model_names:
    evaluate_model(judge_model, model_name, d, evaluate_n)

plot_model_comparison_radar(
    model_names=model_names,
    judge_model=judge_model
)


# What will happen if we choose the range of socre to be 1-5. 1-10. 1-100?
# Let's test it here

In [ ]:
judge_model = "deepseek-v3"
# using the same models as before
for model_name in model_names:
    evaluate_model(judge_model, model_name, d, evaluate_n)


plot_model_comparison_radar(
    model_names=model_names,
    judge_model=judge_model
)

## 4.4 Pairwise Comparison judge, and Bias2: positions.

Another problem is llm's judgement is depend on position.

In [12]:
# here is the prompt for pairwise comparison judge
def compare_model_summaries(model1_name, model2_name, judge_model, num_samples):
    """
    Compare summaries from two different models using a judge model.
    """
    import json, os, random
    from tqdm import tqdm

    model1_file = f"judge_results/{model1_name}_evaluations_{judge_model}.jsonl"
    model2_file = f"judge_results/{model2_name}_evaluations_{judge_model}.jsonl"

    with open(model1_file, "r") as f:
        model1_data = [json.loads(l) for l in f]

    with open(model2_file, "r") as f:
        model2_data = [json.loads(l) for l in f]

    common_ids = list(
        {d["article_id"] for d in model1_data}.intersection(
            {d["article_id"] for d in model2_data}
        )
    )
    if len(common_ids) < num_samples:
        print(f"Warning: only {len(common_ids)} common articles, using all.")
        num_samples = len(common_ids)

    selected_ids = random.sample(common_ids, num_samples)
    m1 = {d["article_id"]: d for d in model1_data}
    m2 = {d["article_id"]: d for d in model2_data}

    wins = {model1_name: 0, model2_name: 0, "tie": 0}

    for aid in tqdm(selected_ids, desc=f"{model1_name} vs {model2_name}"):
        art = m1[aid]["article"]
        ref = m1[aid]["reference"]
        s1  = m1[aid]["model_answer"]
        s2  = m2[aid]["model_answer"]

        prompt = f"""You are an expert judge evaluating two summaries of a news article.

Article: {art}

Reference Summary: {ref}

Summary A: {s1}

Summary B: {s2}

Compare the two summaries on elegance, fluency, conciseness, and creativity.
Which summary is better overall? Respond with **only** "A", "B", or "Tie" on the first line,
then optionally give a short explanation on the next line.
"""

        resp = client.chat.completions.create(
            model=judge_model,
            messages=[{"role": "user", "content": prompt}],
        )

        judge_text = resp.choices[0].message.content.strip()
        first_line = judge_text.splitlines()[0].strip().upper()

        if first_line.startswith("A"):
            wins[model1_name] += 1
        elif first_line.startswith("B"):
            wins[model2_name] += 1
        else:
            wins["tie"] += 1

    print(f"\nResults ({judge_model} as judge)")
    print(f"{model1_name} wins: {wins[model1_name]}")
    print(f"{model2_name} wins: {wins[model2_name]}")
    print(f"Ties: {wins['tie']}")
    return wins

In [ ]:
compare_results = compare_model_summaries(
    "qwen2.5-72b-instruct", 
    "llama-3.3-70b-instruct", 
    "deepseek-v3", 
    evaluate_n
)

compare_results = compare_model_summaries(
    "llama-3.3-70b-instruct",
    "qwen2.5-72b-instruct", 
    "deepseek-v3", 
    evaluate_n
)

See the differences?

## 4.5 Further Questions about LLM-as-Judge

There are several interesting questions about using LLMs as judges that you might want to explore:

1. **Judge Bias**: Does the choice of judge model significantly impact the results? 

2. **Prompt Sensitivity**: How sensitive are the judgments to the specific wording of the prompt? 

3. **Consistency**: How consistent are LLM judges across multiple evaluations of the same content? 

4. **Ability Limitation**: How do models perform when judging the tasks they cannot achieve perfectly?

5. **Correlation with Human Judgment**: How well do LLM judgments align with human preferences? 

6. **Metric Weighting**: Do certain metrics (like fluency or creativity) have more influence on the overall judgment than others?



In [ ]:
#### your task ####
# find a dataset that you are interested in, and use llm-as-judge to evaluate the model.
# plot the radar graph of the results (with dimensions of your choice, but as least 4 dimensions)
# you should find two models with as much performance difference as possible.
# to do so, you need choose a good evaluation set, as well as a good judge model.
